## Week 4 - Part 1: Decsription of the data science problem

#### London. The biggest city in European Union (at least until BREXIT). Great multi-cultural mix that that is very open everyone. Yet this is one of the most common destination of emigration among Poles. As a Pole I saw that many of my friends emigrated to UK, including London, where they started to settle and put down roots. 

#### My friend (of course imaginatory) wants to study and work in London (let's ommit BREXIT stuff, and so on , this is fully imaginatory quest). He/She asked me which part of London should he/she live. As beginner data scientist I've wanted to help my friend with bunch of analysis. My friend has a couple of criteria that I should take into account in my analysis:

- Price of a flat rental
- Public transport accesability
- Safety of cetain area
- General satisfaction of living in certain area
- Green space in neighborhood
- Polish venues in given area

#### My goal is to find an area in London will be the most satisfying for my friend. He/she wants to study and work part time. He/she will commute often therefore area should have great accesibility to public transport. 
#### Moreover my friend is a student, therefore rental costs should not be very high. For time being he/she is prone to rent a single room rather than a small flat. 
#### Area should be quite safe, therefore any dodgy places with high crime rate should be avoided as well. In general the area should be rather cheerful, and people living there should be in general satisfied.
#### As a Pole, my friend is looking also for some Polish venues, like shops or restaurants where he/she can feels like home for a bit. 

#### My friend hasn't specified which of the main characteristics have importance, therefore for the sake of simplicity I assumed that they are equal... Nevertheless I feel that my friend after seeing the results may weight higher some features therefore I would like to create some algorithm that will point area of London according to my friend's preferences.

#### Boroughs are large areas, therefore we will analyse neighbours afterwards. Unfortunatey there are not detailed data about living in certain neighborhoods, therefore after choosing particular borough I'll try to do ML clustering for  neighborhoods in particular borough using data from Forsquare

## Week 4 - Part 2: Description of the data

#### First information is that London is divided to 33 areas - 32 boroughs and 1 separate "City of London". For the sake of simplicity I'll later on call those 33 areas as boroughs.
#### Full list of boroughs with names and their coordinates are available on Wikipedia. On the next step I'll show you the list  

In [1]:
#In order to do so I'll import pandas & numpy and instal lxml which helps with data scrapping
import pandas as pd
import numpy as np


In [3]:
pip install lxml

     |████████████████████████████████| 5.8MB 29.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


##### Now I'm importing data from the wikipedia

In [2]:
London_bor=pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs')

##### And merging boroughs and City

In [3]:
City=London_bor[1]
Boro=London_bor[0]
Boro.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [4]:
City

,‘Borough’,Inner,Status,Local authority,Political control,Headquarters,Area(sq mi),Population(2011 est),Co-ordinates,Nr. inmap
0,City of London,([note 5],Sui generis;City;Ceremonial county,Corporation of London;Inner Temple;Middle Temple,?,Guildhall,1.12,7000,51°30′56″N 0°05′32″W﻿ / ﻿51.5155°N 0.0922°W,1


##### actually I need here only Borough name and coordinates.I must change column name of City toBorough without ' ' 

In [5]:
# Droping spare columns
City.drop(City.columns[[1, 2, 3, 4, 5, 6, 7, 9]], axis=1, inplace=True)
Boro.drop(Boro.columns[[1, 2, 3, 4, 5, 6, 7, 9]], axis=1, inplace=True)

In [6]:
City.columns=['Borough', 'Co-ordinates']

In [7]:
London_borough=pd.concat([City, Boro], axis=0, sort=False).reset_index(drop=True)

In [8]:
London_borough.head()

,Borough,Co-ordinates
0,City of London,51°30′56″N 0°05′32″W﻿ / ﻿51.5155°N 0.0922°W
1,Barking and Dagenham [note 1],51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E
2,Barnet,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
3,Bexley,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
4,Brent,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W


##### We see that some boroughs have some notes in parentesis, therefore I need to clean it

In [9]:
London_borough['Borough']=London_borough['Borough'].str.partition(" [", True) 

In [10]:
London_borough['Co-ordinates']=London_borough['Co-ordinates'].str.partition(" / ", True) 

In [11]:
new=London_borough['Co-ordinates'].str.partition(" ", True) 

In [12]:
London_borough['Latitude']=new[0]
London_borough['Longitude']=new[2]

In [13]:
London_borough.head()

,Borough,Co-ordinates,Latitude,Longitude
0,City of London,51°30′56″N 0°05′32″W﻿,51°30′56″N,0°05′32″W﻿
1,Barking and Dagenham,51°33′39″N 0°09′21″E﻿,51°33′39″N,0°09′21″E﻿
2,Barnet,51°37′31″N 0°09′06″W﻿,51°37′31″N,0°09′06″W﻿
3,Bexley,51°27′18″N 0°09′02″E﻿,51°27′18″N,0°09′02″E﻿
4,Brent,51°33′32″N 0°16′54″W﻿,51°33′32″N,0°16′54″W﻿


In [14]:
London_borough=London_borough.drop(columns=['Co-ordinates'])

In [15]:
##### Then I'll step to downloading data of a flat rentals 

In [16]:
pip install xlrd

     |████████████████████████████████| 112kB 30.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
from pandas import DataFrame, read_csv

In [18]:
file='https://data.london.gov.uk/download/average-private-rents-borough/73b9fb07-b5bb-4a53-88b7-c17269879a08/voa-average-rent-borough.xls'
prices=pd.read_excel(file, sheet_name='Raw data', skiprows=2)

In [19]:
prices.head()

,Year,Quarter,Code,Area,Category,Count of rents,Average,Lower quartile,Median,Upper quartile
0,2011,Q2,E09000001,City of London,Room,-,-,-,-,-
1,2011,Q2,E09000002,Barking and Dagenham,Room,92,336,282,347,390
2,2011,Q2,E09000003,Barnet,Room,945,450,399,433,500
3,2011,Q2,E09000004,Bexley,Room,119,390,347,390,433
4,2011,Q2,E09000005,Brent,Room,344,469,390,457,550


In [20]:
# I'm keeping newest obs (2019 - only Q1 is available), and Median
prices.drop(prices.columns[[2, 5, 6, 7, 9]], axis=1, inplace=True)

In [21]:
prices=prices[prices['Year'] == 2019].reset_index(drop=True)

In [22]:
prices.head()

,Year,Quarter,Area,Category,Median
0,2019,Q1,City of London,Room,.
1,2019,Q1,Barking and Dagenham,Room,650
2,2019,Q1,Barnet,Room,588
3,2019,Q1,Bexley,Room,585
4,2019,Q1,Brent,Room,602


In [42]:
prices.Category.unique()

array(['Room', 'Studio', 'One Bedroom', 'Two Bedroom', 'Three Bedroom',
       'Four or more Bedrooms', 'All categories'], dtype=object)

#### there are few categories. My colleague will look for the cheapest type i.e. Room
#### Let's look before on data types. And then delete some rows without any observations, and set Median as a float

In [23]:
print(prices.dtypes)

Year         int64
Quarter     object
Area        object
Category    object
Median      object
dtype: object


In [24]:
#deleting "." in median, which means no obs
prices.drop(prices[prices.Median=='.'].index, inplace=True)
prices.drop(prices[prices.Median=='..'].index, inplace=True)

In [25]:
prices['Median']=prices['Median'].astype(float)

In [26]:
print(prices.dtypes)

Year          int64
Quarter      object
Area         object
Category     object
Median      float64
dtype: object


In [43]:
prices.Area.unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
       'Redbridge', 'Southwark', 'Sutton', 'Tower Hamlets',
       'Waltham Forest', 'Wandsworth', 'Westminster', 'City of London',
       'Richmond upon Thames'], dtype=object)

#### There are some observations like LONDON which are spare for me. I need only Boroughs

In [27]:
prices=prices[prices.Area.isin(London_borough['Borough'])]

In [41]:
#checking if there are 33 unique boroughs  
prices.describe()

,Year,Median
count,227.0,227.000000
mean,2019.0,1496.594714
std,0.0,880.468788
min,2019.0,500.000000
25%,2019.0,950.000000
50%,2019.0,1338.000000
75%,2019.0,1787.500000
max,2019.0,8450.000000


#### It seems that diversity of prices in London is very wide. We have "analysed" here all types of rentals. My friend will be looking only for single room, therefore I'll put below prices of just a room rentals. 

In [45]:
prices[prices.Category=='Room'].sort_values('Median')

,Year,Quarter,Area,Category,Median
7,2019,Q1,Croydon,Room,500.0
25,2019,Q1,Redbridge,Room,528.0
28,2019,Q1,Sutton,Room,550.0
22,2019,Q1,Lewisham,Room,550.0
9,2019,Q1,Enfield,Room,550.0
20,2019,Q1,Kingston upon Thames,Room,550.0
30,2019,Q1,Waltham Forest,Room,573.0
14,2019,Q1,Harrow,Room,575.0
3,2019,Q1,Bexley,Room,585.0
5,2019,Q1,Bromley,Room,585.0


##### Cool, but this is only just a one single data that is needed to full analysis

### Now I'm moving to public transport accesibility, safety and general satisfaction of living

In [30]:
file='https://data.london.gov.uk/dataset/london-borough-profiles/resource/80647ce7-14f3-4e31-b1cd-d5f7ea3553be/download'
Borough_info=pd.read_excel(file, sheet_name='Data')


In [31]:
Borough_info=Borough_info[Borough_info['Area name'].isin(London_borough['Borough'])]

In [32]:
Borough_info.head()

,Code,New code,Area name,Inner/ Outer London,GLA Population Estimate 2017,GLA Household Estimate 2017,Inland Area (Hectares),Population density (per hectare) 2017,"Average Age, 2017","Proportion of population aged 0-15, 2015",...,Happiness score 2011-14 (out of 10),Anxiety score 2011-14 (out of 10),Childhood Obesity Prevalance (%) 2015/16,People aged 17+ with diabetes (%),Mortality rate from causes considered preventable 2012/14,Political control in council,Proportion of seats won by Conservatives in 2014 election,Proportion of seats won by Labour in 2014 election,Proportion of seats won by Lib Dems in 2014 election,Turnout at 2014 local elections
1,E09000001,E09000001,City of London,Inner London,8800.0,5325.82,290.393,30.3037,43.2,11.4,...,5.99,5.57,NaN,2.637945,128.833,.,.,.,.,.
2,E09000002,E09000002,Barking and Dagenham,Outer London,209000.0,78188.4,3610.78,57.8822,32.9,27.2,...,7.05,3.05,28.542,7.272108,227.605,Lab,0,100,0,36.4974
3,E09000003,E09000003,Barnet,Outer London,389600.0,151423,8674.83,44.9115,37.3,21.1,...,7.37,2.75,20.6579,6.026084,133.77,Cons,50.7937,.,1.5873,40.481
4,E09000004,E09000004,Bexley,Outer London,244300.0,97735.8,6058.07,40.3264,39.0,20.6,...,7.21,3.29,22.7093,6.859383,164.286,Cons,71.4286,23.8095,0,39.6308
5,E09000005,E09000005,Brent,Outer London,332100.0,121048,4323.26,76.817,35.6,20.9,...,7.22,2.92,24.3444,7.920348,169.356,Lab,9.52381,88.8889,1.5873,36.272


#### NOw i'm choosing only few columns - area name, population & size & age (dunno why, I like statistics), crime rate, greenspace and happiness score

In [33]:
Borough_info=Borough_info[['Area name', 'GLA Population Estimate 2017', 'Inland Area (Hectares)', 'Average Age, 2017', 'Crime rates per thousand population 2014/15', '% of area that is Greenspace, 2005', 'Average Public Transport Accessibility score, 2014', 'Happiness score 2011-14 (out of 10)']]

In [34]:
Borough_info.columns=['Area', 'Population', 'Size (ha)', 'Avg. Age', 'Crime rate', 'Greenspace', 'Public transport accessibility', 'Happiness score']

In [35]:
print(Borough_info.dtypes)

Area                               object
Population                        float64
Size (ha)                          object
Avg. Age                          float64
Crime rate                         object
Greenspace                         object
Public transport accessibility     object
Happiness score                   float64
dtype: object


##### And now I'm turnning dtypes to float. I need to replace '.' value in London City. I set it as an average 

In [36]:
Borough_info.replace(".", np.nan, inplace = True)
avg_Crime = Borough_info['Crime rate'].mean(axis=0)
Borough_info['Crime rate'].replace(np.nan, avg_Crime, inplace=True)

In [37]:
Borough_info['Size (ha)']=Borough_info['Size (ha)'].astype(float)
Borough_info['Greenspace']=Borough_info['Greenspace'].astype(float)
Borough_info['Public transport accessibility']=Borough_info['Public transport accessibility'].astype(float)
Borough_info['Crime rate']=Borough_info['Crime rate'].astype(float)

In [40]:
#checking if there are 33 unique boroughs  
Borough_info.describe()

,Population,Size (ha),Avg. Age,Crime rate,Greenspace,Public transport accessibility,Happiness score
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,267739.393939,4764.082261,36.375758,84.873250,31.412121,3.943296,7.203636
std,75383.345058,3270.913690,2.487849,30.642463,12.673120,1.306933,0.249472
min,8800.000000,290.393400,31.400000,50.396101,4.800000,2.363094,5.990000
25%,231200.000000,2681.003500,35.000000,64.133577,23.200000,2.971488,7.160000
50%,276200.000000,3762.473800,36.200000,78.010662,31.700000,3.443409,7.210000
75%,321000.000000,5641.899700,37.700000,99.646824,38.200000,4.874750,7.310000
max,389600.000000,15013.489200,43.200000,212.413100,59.300000,7.862302,7.560000


##### Crime rate is measured as crimes per thousand inhabitants. 
##### Greenspace is % of 'green' areas in given borough
##### Public transport accesability is measured from 0 to 10, where 10 means great accesability
##### Happiness is as well measured from 0 to 10, where 10 is the biggest score 

#### Next I'll use Forsquare to find Polish venues in the vicinity of the Borough center. I will not give the code here,. It will be uploaded in next week's submission. My goal would be there to find all venues that have words 'Polish', 'Poland', 'Polski' in their names or word 'Polish' in Venue type. Result will be total numbers of Polish venues.